## Julia version 1.8.1

## Code blocks intended to be run independently

# Figure 2

In [ ]:
using DifferentialEquations
using Plots

In [ ]:
#define model with delay
function dde_system(du,u,h,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
    histC_MDSC = h(p, t-τ; idxs=1)
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*histC_MDSC/(γ₁+(histC_MDSC)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define model without delay
function ode_system(du,u,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define initial number of tumor cells for delay model
h(p, t; idxs=nothing) = typeof(idxs) <: Number ? 1.0 : ones(4)
h(1,0)

In [ ]:
#set model parameters
α₂ = 10.0*(10.0^1.0) #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#set numerical solution algorithm
alg_ODE = Tsit5()

In [ ]:
#set numerical solution algorithm
alg_DDE = MethodOfSteps(Tsit5())

## Panel A

In [ ]:
#solve with τ = 0
prob_panel_a = ODEProblem(ode_system,u0,tspan,p)
sol_panel_a = solve(prob_panel_a,alg_ODE)

## Panel B

In [ ]:
#solve with τ = 10
τ = 10.0
lags = [τ]
prob_panel_b = DDEProblem(dde_system,u0,h,tspan,p;constant_lags=lags)
sol_panel_b = solve(prob_panel_b,alg_DDE)

## Panel C

In [ ]:
#solve with τ = 50
τ = 50.0
lags = [τ]
prob_panel_c = DDEProblem(dde_system,u0,h,tspan,p;constant_lags=lags)
sol_panel_c = solve(prob_panel_c,alg_DDE)

## Panel D

In [ ]:
#solve with τ = 365
τ = 365.0
lags = [τ]
prob_panel_d = DDEProblem(dde_system,u0,h,tspan,p;constant_lags=lags)
sol_panel_d = solve(prob_panel_d,alg_DDE)

## Plot

In [ ]:
#plot results
l = @layout [a b; c d]
logocolors = Colors.JULIA_LOGO_COLORS
a = plot(sol_panel_a, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 0",titlefontsize=25)
b = plot(sol_panel_b, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 10",titlefontsize=25)
c = plot(sol_panel_c, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 50",titlefontsize=25)
d = plot(sol_panel_d, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 365",titlefontsize=25)
plot(a, b, c, d, layout = l,size =(1600,1200),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),plot_title="Effect of MDSC delay τ", font = "Ariel",titlefontsize=25)

# Figure S1

In [ ]:
using DifferentialEquations
using Plots

In [ ]:
#define model with delay
function dde_system(du,u,h,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
    histC_CTL = h(p, t-τ₁; idxs=1)
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*histC_CTL*NK + α₇*(histC_CTL^2)/(γ₃+(histC_CTL^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define model without delay
function ode_system(du,u,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define initial number of tumor cells for delay model
h(p, t; idxs=nothing) = typeof(idxs) <: Number ? 1.0 : ones(4)
h(1,0)

In [ ]:
#set model parameters
α₂ = 10.0*(10.0^1.0) #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#set numerical solution algorithm
alg_ODE = Tsit5()

In [ ]:
#set numerical solution algorithm
alg_DDE = MethodOfSteps(Tsit5())

## Panel A

In [ ]:
#solve with τ₁ = 0
prob_panel_a = ODEProblem(ode_system,u0,tspan,p)
sol_panel_a = solve(prob_panel_a,alg_ODE)

## Panel B

In [ ]:
#solve with τ₁ = 10
τ₁ = 10.0
lags = [τ₁]
prob_panel_b = DDEProblem(dde_system,u0,h,tspan,p;constant_lags=lags)
sol_panel_b = solve(prob_panel_b,alg_DDE)

## Panel C

In [ ]:
#solve with τ₁ = 50
τ₁ = 50.0
lags = [τ₁]
prob_panel_c = DDEProblem(dde_system,u0,h,tspan,p;constant_lags=lags)
sol_panel_c = solve(prob_panel_c,alg_DDE)

## Panel D

In [ ]:
#solve with τ₁ = 365
τ₁ = 365.0
lags = [τ₁]
prob_panel_d = DDEProblem(dde_system,u0,h,tspan,p;constant_lags=lags)
sol_panel_d = solve(prob_panel_d,alg_DDE)

## Plot

In [ ]:
#plot results
l = @layout [a b; c d]
logocolors = Colors.JULIA_LOGO_COLORS
a = plot(sol_panel_a, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ₁ = 0",titlefontsize=25)
b = plot(sol_panel_b, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ₁ = 10",titlefontsize=25)
c = plot(sol_panel_c, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ₁ = 50",titlefontsize=25)
d = plot(sol_panel_d, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ₁ = 365",titlefontsize=25)
plot(a, b, c, d, layout = l,size =(1600,1200),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),plot_title="Effect of CTL delay τ₁", font = "Ariel",titlefontsize=25)

# Figure 3A-C

In [ ]:
using DifferentialEquations
using Plots

In [ ]:
#define model without delay
function ode_system(du,u,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#set numerical solution algorithm
alg_ODE = Tsit5()

## Panel A

In [ ]:
#set model parameters
α₂ = 100.0 #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate


γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#calculate growth condition G
growthcondition = α₁*log(η)-β₁*(ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃))-ζ₁

In [ ]:
#solve with ζ₁ = 0
prob_panel_a = ODEProblem(ode_system,u0,tspan,p)
sol_panel_a = solve(prob_panel_a,alg_ODE)

## Panel B

In [ ]:
#set model parameters
α₂ = 100.0 #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.81 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate


γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#calculate growth condition G
growthcondition = α₁*log(η)-β₁*(ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃))-ζ₁

In [ ]:
#solve with ζ₁ = 0.81
prob_panel_b = ODEProblem(ode_system,u0,tspan,p)
sol_panel_b = solve(prob_panel_b,alg_ODE)

## Panel C

In [ ]:
#set model parameters
α₂ = 100.0 #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 1.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate


γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#calculate growth condition G
growthcondition = α₁*log(η)-β₁*(ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃))-ζ₁

In [ ]:
#solve with ζ₁ = 1.0
prob_panel_c = ODEProblem(ode_system,u0,tspan,p)
sol_panel_c = solve(prob_panel_c,alg_ODE)

In [ ]:
#plot results
l = @layout [a b c]
logocolors = Colors.JULIA_LOGO_COLORS
a = plot(sol_panel_a, lw = 4, yaxis=:log,size =(1600/3,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "G = 0.8",titlefontsize=25)
b = plot(sol_panel_b, lw = 4, yaxis=:log,size =(1600/3,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "G = 0",titlefontsize=25)
c = plot(sol_panel_c, lw = 4, yaxis=:log,size =(1600/3,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "G = -0.2",titlefontsize=25)
plot(a, b, c, layout = l,size =(1600,600),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),plot_title="Effect of tumor growth threshold G", font = "Ariel",titlefontsize=25)

# Figure 3E-F

In [ ]:
using DifferentialEquations
using Plots

In [ ]:
#define model without delay
function ode_system(du,u,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#set numerical solution algorithm
alg_ODE = Tsit5()

## Panel E

In [ ]:
#set model parameters
α₂ = 100.0 #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = (10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate


γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#calculate growth condition G
growthcondition = α₁*log(η)-β₁*(ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃))-ζ₁

In [ ]:
#solve with β₃ = 10⁻⁵
prob_panel_e = ODEProblem(ode_system,u0,tspan,p)
sol_panel_e = solve(prob_panel_e,alg_ODE)

## Panel F

In [ ]:
#set model parameters
α₂ = 100.0 #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = (10.0^(-4.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate


γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#calculate growth condition G
growthcondition = α₁*log(η)-β₁*(ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃))-ζ₁

In [ ]:
#solve with β₃ = 10⁻⁴
prob_panel_f = ODEProblem(ode_system,u0,tspan,p)
sol_panel_f = solve(prob_panel_f,alg_ODE)

In [ ]:
#plot results
l = @layout [e f]
logocolors = Colors.JULIA_LOGO_COLORS
e = plot(sol_panel_e, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "β₃ = 10⁻⁵",titlefontsize=25)
f = plot(sol_panel_f, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "β₃ = 10⁻⁴",titlefontsize=25)
plot(e, f, layout = l,size =(1600,600),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),plot_title="Effect of NK cells inhibition rate by MDSCs β₃", font = "Ariel",titlefontsize=25)

# Figure 3D

In [ ]:
using DiffEqSensitivity
using Statistics
using DiffEqCallbacks
using DifferentialEquations
using Plots
using GlobalSensitivity
import Random

In [ ]:
#define condition for simulation to end
function condition(u,t,integrator) 
  u[1] < 0.01 || u[2] < 0.0 || u[3] < 0.0 || u[4] < 0.0
end
floor_event = DiscreteCallback(condition, terminate!)

In [ ]:
#define model without delay
function model(du,u,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄,γ₁,γ₂,γ₃ = p
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(max(η/T,1)) - β₁*T*NK - β₂*T*CTL - ζ₁*T 
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

α₂ = 10.0*(10.0^1.0) #MDSCs production rate 
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄,γ₁,γ₂,γ₃)
tspan = (0.0,1000.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]
prob = ODEProblem(model,u0,tspan,p)
prob_SS = SteadyStateProblem(prob)

In [ ]:
#set up function for sensitivity analysis
f = function (p)
  prob_func(prob,i,repeat) = remake(prob;p=p[:,i])
  ensemble_prob = EnsembleProblem(prob,prob_func=prob_func)
    sol = solve(ensemble_prob,Tsit5(),maxiters=10^8,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u),EnsembleThreads(),trajectories=size(p,2))
  out = zeros(2,size(p,2))
  for i in 1:size(p,2)
    out[1,i] = last(sol[i][1,:])
    out[2,i] = maximum(sol[i][1,:]) 
  end
  out
end

In [ ]:
#perform sensitivity analysis (with set random seed for reproducibility)
Random.seed!(1200)
m = gsa(f,Morris(total_num_trajectory=200,num_trajectory=20),[[0.0,10.0^(3.0)],[10.0^(7.0),10.0^(9.0)],[0.0,1.0],[10.0^(-2.0),5*10.0^(-1.0)],[10.0^(6.0),10.0^(8.0)],[10.0^(-7.0),10.0^(-6.0)],[10.0^(-7.0),10.0^(-6.0)],[0.0,0.1],[10.0^(3.0),10.0^(5.0)],[10.0^(-2.0),10.0^(-1.0)],[10.0^(-5.0),10.0^(-4.0)],[10.0^(-2.0),10.0^(-1.0)],[10.0^(-7.0),10.0^(-6.0)],[5*10.0^(-2.0),10.0^(-1.0)],[5*10.0^(-5.0),5*10.0^(-4.0)],[10.0^(-2.0),10.0^(-1.0)],[10.0^(9.0),10.0^(11.0)],[10.0^(6.0),10.0^(8.0)],[10.0^(6.0),10.0^(8.0)]],batch=true)

In [ ]:
#sensitivity analysis values
m.means

In [ ]:
#sensitivity analysis values
means1 = m.means[1,:]

In [ ]:
#sensitivity analysis values
means2 = m.means[2,:]

In [ ]:
#sensitivity analysis values
m.variances

In [ ]:
#use absolute value for plotting
for i = 1:length(means1)
   means1[i] = abs(means1[i]) 
   means2[i] = abs(means2[i]) 
end

In [ ]:
#sensitivity analysis values
means1

In [ ]:
#sensitivity analysis values
means2

In [ ]:
#plot results
l = @layout [a b]
plot1 = scatter(means1, m.variances[1,:],series_annotations=[:α₂,:α₃,:ζ₂,:α₁,:η,:β₁,:β₂,:ζ₁,:α₄,:α₅,:β₃,:ζ₃,:α₆,:α₇,:β₄,:ζ₄,:γ₁,:γ₂,:γ₃],seriescolor=[:yellow :red :green],xscale=:log10,yscale=:log10,markersize=20,legend=false, title = "Steady state of tumor population", xlabel = "Mean", ylabel = "Variance")
plot2 = scatter(means2, m.variances[2,:],series_annotations=[:α₂,:α₃,:ζ₂,:α₁,:η,:β₁,:β₂,:ζ₁,:α₄,:α₅,:β₃,:ζ₃,:α₆,:α₇,:β₄,:ζ₄,:γ₁,:γ₂,:γ₃],color=:yellow,xscale=:log10,yscale=:log10,markersize=20,legend=false, title = "Maximum of tumor population", xlabel = "Mean", ylabel = "Variance")
plot(plot1, plot2, layout = l,size =(1600,600),xtickfont=font(18),ytickfont=font(18),guidefont=font(25),legendfont=font(25),titlefont=font(25))

In [ ]:
#plot results
plot_justmean = scatter(["α₂","α₃","ζ₂","α₁","η","β₁","β₂","ζ₁","α₄","α₅","β₃","ζ₃","α₆","α₇","β₄","ζ₄","γ₁","γ₂","γ₃"],means1,color=[:lightgreen, :lightgreen, :red, :lightgreen, :lightgreen, :red, :red, :red, :red, :red, :lightgreen, :lightgreen, :red, :red, :lightgreen, :lightgreen, :red, :lightgreen, :lightgreen],yscale=:log10,markersize=20,legend=false, title = "", xlabel = "Parameter", ylabel = "Morris mean") #mean of tumor population
plot(plot_justmean,size =(1600,600),xtickfont=font(18),ytickfont=font(18),guidefont=font(25),legendfont=font(25),titlefont=font(25), xticks = :all)

In [ ]:
#plot results
vector_1 = ["α₆", "β₁", "β₂", "β₃", "β₄", "ζ₁", "α₁", "ζ₃", "ζ₂", "ζ₄", "α₂", "α₅", "α₄", "α₇", "η", "α₃", "γ₁", "γ₃", "γ₂"]
vector_2 = [6.60E+11,3.83E+11,2.51E+11,1.55E+10,6.60E+08,1.95E+08,6.98E+07,1.02E+07,2.78E+06,26002.62771,103.8344697,11.91689027,11.61676075,4.113746122,0.629535045,0.082723438,9.69E-06,3.86E-12,3.44E-12]

non_MDSC_size = 30
MDSC_size = 40

non_MDSC_alpha = 0.6
MDSC_alpha = 1

non_MDSC_strokewidth = 1
MDSC_strokewidth = 2

color_positive = [95,177,42]
color_negative = [226,58,52]

logocolors = Colors.JULIA_LOGO_COLORS

vector_size = [non_MDSC_size,non_MDSC_size,non_MDSC_size,MDSC_size,MDSC_size,non_MDSC_size,non_MDSC_size,non_MDSC_size,MDSC_size,non_MDSC_size,MDSC_size,non_MDSC_size,non_MDSC_size,non_MDSC_size,non_MDSC_size,MDSC_size,MDSC_size,non_MDSC_size,non_MDSC_size]
vector_alpha = [non_MDSC_alpha,non_MDSC_alpha,non_MDSC_alpha,MDSC_alpha,MDSC_alpha,non_MDSC_alpha,non_MDSC_alpha,non_MDSC_alpha,MDSC_alpha,non_MDSC_alpha,MDSC_alpha,non_MDSC_alpha,non_MDSC_alpha,non_MDSC_alpha,non_MDSC_alpha,MDSC_alpha,MDSC_alpha,non_MDSC_alpha,non_MDSC_alpha]
vector_color = [logocolors.red, logocolors.red, logocolors.red, logocolors.green, logocolors.green, logocolors.red, logocolors.green, logocolors.green, logocolors.red, logocolors.green, logocolors.green, logocolors.red, logocolors.red, logocolors.red, logocolors.green, logocolors.green, logocolors.red, logocolors.green, logocolors.green]
vector_strokewidth = [non_MDSC_strokewidth,non_MDSC_strokewidth,non_MDSC_strokewidth,MDSC_strokewidth,MDSC_strokewidth,non_MDSC_strokewidth,non_MDSC_strokewidth,non_MDSC_strokewidth,MDSC_strokewidth,non_MDSC_strokewidth,MDSC_strokewidth,non_MDSC_strokewidth,non_MDSC_strokewidth,non_MDSC_strokewidth,non_MDSC_strokewidth,MDSC_strokewidth,MDSC_strokewidth,non_MDSC_strokewidth,non_MDSC_strokewidth]
vector_shape = [:circle, :circle, :circle, :hexagon, :hexagon, :circle, :circle, :circle, :hexagon, :circle, :hexagon, :circle, :circle, :circle, :circle, :hexagon, :hexagon, :circle, :circle]

plot_justmean = scatter(vector_1,vector_2,color=vector_color,yscale=:log10,markersize=vector_size, alpha = vector_alpha, markerstrokewidth = vector_strokewidth, markershape = vector_shape, legend=false, title = "", xlabel = "Parameter", ylabel = "Mean", ylims = [10^(-12), 10^13])
plot(plot_justmean,size =(1600,600),font = "Ariel",xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),titlefont=font(25),xticks = :all,plot_title="Morris global parameter sensitivity analysis",titlefontsize=25)

# Figures 4A,C and S2

In [ ]:
using DifferentialEquations
using Plots
using StochasticDelayDiffEq
using DiffEqCallbacks
using Statistics
import Random

In [ ]:
#define model with delay
function dde_system(du,u,h,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
    histC_MDSC = h(p, t-τ; idxs=1)
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*histC_MDSC/(γ₁+(histC_MDSC)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define model without delay
function ode_system(du,u,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define initial number of tumor cells for delay model
h(p, t; idxs=nothing) = typeof(idxs) <: Number ? 2.0 : ones(4).+ 1
h(1,0)

In [ ]:
#set model parameters
α₂ = 10.0*(10.0^1.0) #MDSCs production
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [2.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#define stochastic noise
W = WienerProcess(0.0,0.0,0.0)

function σfunction(du,u,h,p,t)
  du[1] = u[1]
  du[2] = u[2]
  du[3] = u[3]
  du[4] = u[4]
end

function σfunction_nodelay(du,u,p,t)
  du[1] = u[1]
  du[2] = u[2]
  du[3] = u[3]
  du[4] = u[4]
end

In [ ]:
#define conditions for simulation to end
function condition(u,t,integrator) 
  u[1] < 1.0
end

function floor_aff!(integrator)
    integrator.u[1]=1.0
end

function terminate_affect!(integrator)
    terminate!(integrator)
end

floor_event = DiscreteCallback(condition, terminate!)
floor_event2 = DiscreteCallback(condition, floor_aff!)

## Panel A

In [ ]:
#solve with τ = 0 (with set random seed for reproducibility)
runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1240)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS

prob_nodelay = SDEProblem(ode_system,σfunction_nodelay,u0,tspan,p,noise=W)
sol = solve(prob_nodelay,SOSRI(),callback=floor_event)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    a = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 0",titlefontsize=25) 

    display(a)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end
    plot1 = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 0",titlefontsize=25)
    display(plot1)
end
    
print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Panel B

In [ ]:
#solve with τ = 10 (with set random seed for reproducibility)
τ = 10.0
lags = [τ]

print(τ, "\n")

runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1240)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS
    
prob1 = SDDEProblem(dde_system,σfunction,u0,h,tspan,p; constant_lags=lags,noise=W)
sol = solve(prob1,SOSRI(),maxiters=10^8,callback=floor_event2)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    b = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 10",titlefontsize=25) 

    display(b)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end
    plot1 = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 10",titlefontsize=25)
    display(plot1)
end

print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Panel C

In [ ]:
#solve with τ = 50 (with set random seed for reproducibility)
τ = 50.0
lags = [τ]

print(τ, "\n")

runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1240)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS

prob1 = SDDEProblem(dde_system,σfunction,u0,h,tspan,p; constant_lags=lags,noise=W)
sol = solve(prob1,SOSRI(),maxiters=10^8,callback=floor_event2)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    c = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 50",titlefontsize=25) 

    display(c)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end
    plot1 = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 50",titlefontsize=25)
    display(plot1)
end

print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Panel D

In [ ]:
#solve with τ = 365 (with set random seed for reproducibility)
τ = 365.0
lags = [τ]

print(τ, "\n")

runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1256)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS
 
prob1 = SDDEProblem(dde_system,σfunction,u0,h,tspan,p; constant_lags=lags,noise=W)
sol = solve(prob1,SOSRI(),maxiters=10^8,callback=floor_event2)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    d = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 365",titlefontsize=25) 

    display(d)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end
    plot1 = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 365",titlefontsize=25)
    display(plot1)
end


print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Plot

In [ ]:
#plot results
l = @layout [a b; c d]
logocolors = Colors.JULIA_LOGO_COLORS
plot(a, b, c, d, layout = l,size =(1600,1200),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),plot_title="Successful tumors", font = "Ariel",titlefontsize=25)

# Figures 4B,D and S3

In [ ]:
using DifferentialEquations
using Plots
using StochasticDelayDiffEq
using DiffEqCallbacks
using Statistics 
import Random

In [ ]:
#define model with delay
function dde_system(du,u,h,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
    histC_MDSC = h(p, t-τ; idxs=1)
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*histC_MDSC/(γ₁+(histC_MDSC)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define model without delay
function ode_system(du,u,p,t)
    T,MDSC,NK,CTL = u
    α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄ = p
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(η/T) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#define initial number of tumor cells for delay model
h(p, t; idxs=nothing) = typeof(idxs) <: Number ? 2.0 : ones(4).+ 1
h(1,0)

In [ ]:
#set model parameters
α₂ = 10.0*(10.0^1.0) #MDSCs production
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄)
tspan = (0.0,365.0)
u0 = [2.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
#define stochastic noise
W = WienerProcess(0.0,0.0,0.0)

function σfunction(du,u,h,p,t)
  du[1] = u[1]
  du[2] = u[2]
  du[3] = u[3]
  du[4] = u[4]
end

function σfunction_nodelay(du,u,p,t)
  du[1] = u[1]
  du[2] = u[2]
  du[3] = u[3]
  du[4] = u[4]
end

In [ ]:
#define conditions for simulation to end
function condition(u,t,integrator) 
  u[1] < 1.0
end

function floor_aff!(integrator)
    integrator.u[1]=1.0
end

function terminate_affect!(integrator)
    terminate!(integrator)
end

floor_event = DiscreteCallback(condition, terminate!)
floor_event2 = DiscreteCallback(condition, floor_aff!)

## Panel A

In [ ]:
#solve with τ = 0 (with set random seed for reproducibility)
runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1234)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS

prob_nodelay = SDEProblem(ode_system,σfunction_nodelay,u0,tspan,p,noise=W)
sol = solve(prob_nodelay,SOSRI(),callback=floor_event)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    plot1 = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 0",titlefontsize=25) 

    display(plot1)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end

    for i in sol.t[BB]:0.1:10.0
        push!(sol.t,i)
        push!(sol,sol[:,BB])
    end

    a = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 0",titlefontsize=25)
    a_extended = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,10.0), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 0",titlefontsize=25)
    display(a)
    display(a_extended)
end
    
print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Panel B

In [ ]:
#solve with τ = 10 (with set random seed for reproducibility)
τ = 10.0
lags = [τ]

print(τ, "\n")

runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1230)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS

prob1 = SDDEProblem(dde_system,σfunction,u0,h,tspan,p; constant_lags=lags,noise=W)
sol = solve(prob1,SOSRI(),maxiters=10^8,callback=floor_event2)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    plot1 = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 10",titlefontsize=25) 

    display(plot1)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end

    time_vector = sol.t[1:BB]   
    sol1 = sol[:,1:BB]
    for i in sol.t[BB]:0.1:10.0
        push!(time_vector,i)
        sol1 = [sol1 sol[:,BB]]
    end

    b = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 10",titlefontsize=25)
    b_extended = plot(time_vector, transpose(sol1), lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,10.0), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 10",titlefontsize=25)

    display(b)
    display(b_extended)
end

print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Panel C

In [ ]:
#solve with τ = 50 (with set random seed for reproducibility)
τ = 50.0
lags = [τ]

print(τ, "\n")

runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1260)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS
  
prob1 = SDDEProblem(dde_system,σfunction,u0,h,tspan,p; constant_lags=lags,noise=W)
sol = solve(prob1,SOSRI(),maxiters=10^8,callback=floor_event2)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    plot1 = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 50",titlefontsize=25) 

    display(plot1)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end

    time_vector = sol.t[1:BB]   
    sol1 = sol[:,1:BB]
    for i in sol.t[BB]:0.1:10.0
        push!(time_vector,i)
        sol1 = [sol1 sol[:,BB]]
    end

    c = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 50",titlefontsize=25)
    c_extended = plot(time_vector, transpose(sol1), lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,10.0), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 50",titlefontsize=25)

    display(c)
    display(c_extended)
end

print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Panel D

In [ ]:
#solve with τ = 365 (with set random seed for reproducibility)
τ = 365.0
lags = [τ]

print(τ, "\n")

runs = 1.0
tumorsuccess = 0.0
threshold = 1.0

Random.seed!(1240)

meanofsuccessfulltumors = 0.0;

logocolors = Colors.JULIA_LOGO_COLORS

prob1 = SDDEProblem(dde_system,σfunction,u0,h,tspan,p; constant_lags=lags,noise=W)
sol = solve(prob1,SOSRI(),maxiters=10^8,callback=floor_event2)

A = minimum(sol[1,:])    

if A ≥ threshold
    tumorsuccess = tumorsuccess + 1
    meanofsuccessfulltumors = meanofsuccessfulltumors + mean(sol[1,:])
    plot1 = plot(sol, lw = 4, yaxis=:log, size =(800,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 365",titlefontsize=25) 

    display(plot1)
else
    for j in 1:length(sol[1,:])
        global BB = j
        if sol[1,j] < threshold
            break
        end
    end

     time_vector = sol.t[1:BB]   
    sol1 = sol[:,1:BB]
    for i in sol.t[BB]:0.1:365.0
        push!(time_vector,i)
        sol1 = [sol1 sol[:,BB]]
    end

    d = plot(sol, lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,sol.t[BB]), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 365",titlefontsize=25)
    d_extended = plot(time_vector, transpose(sol1), lw = 4, yaxis=:log,size =(800,600), font = "Ariel", xlims = (0,365.0), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.2 0.2], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "τ = 365",titlefontsize=25)

    display(d)
    display(d_extended)
end

print(100*tumorsuccess/runs, "\n")

print(meanofsuccessfulltumors/tumorsuccess)

## Plot

In [ ]:
#plot results
l = @layout [a b; c d]
logocolors = Colors.JULIA_LOGO_COLORS
plot(a, b, c, d, layout = l,size =(1600,1200),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),plot_title="Unsuccessful tumors", font = "Ariel",titlefontsize=25)

In [ ]:
#plot results
l = @layout [a b; c d]
logocolors = Colors.JULIA_LOGO_COLORS
plot(a_extended, b_extended, c_extended, d_extended, layout = l,size =(1600,1200),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),plot_title="Unsuccessful tumors", font = "Ariel",titlefontsize=25)

# Figures 5, S7, and S8

In [ ]:
using Plots

In [ ]:
#plot for α₂
l = @layout [a b; c d; e f]
delays = [0,0.5,1,2,10,50,100,365]

prob_success200 = [0.52067,0.51936,0.51802,0.51573,0.51987,0.51813,0.51899,0.5206]
prob_sigma200 = [0.499575,0.499628,0.499678,0.499755,0.499608,0.499674,0.499642,0.499578]
mean200 = [390917,325657,311041,314798,179711,90063.8,52208,13398.2]
mean_sigma200 = [6.57092*10^6,4.59642*10^6,3.87981*10^6,4.90952*10^6,2.29257*10^6,3.62887*10^6,1.01787*10^6,276487]
number_of_sims200 = [sqrt(100000),sqrt(100000),sqrt(100000),sqrt(100000),sqrt(100000),sqrt(100000),sqrt(100000),sqrt(100000)]
time200 = [1.8359,1.79055,1.87445,1.8974,1.93117,1.88062,1.96586,2.14636]
time_sigma200 =[7.96526,7.4512,8.39377,8.61256,9.12563,8.11352,9.15333,11.7485]

prob_success150 = [0.440281,0.442417,0.440206,0.441288,0.435629,0.428312,0.419405,0.379864]
prob_sigma150 = [0.496423,0.496676,0.496414,0.496543,0.495841,0.494836,0.493464,0.485355]
mean150 = [297352,254711,249961,238555,135733,57244.9,39425.7,11631.9]
mean_sigma150 = [5.20893*10^6,4.16001*10^6,3.57743*10^6,5.68132*10^6,1.59068*10^6,1.02464*10^6,1.32366*10^6,190806]
number_of_sims150 = [sqrt(120230),sqrt(104311),sqrt(105696),sqrt(104816),sqrt(105459),sqrt(108540),sqrt(110218),sqrt(126453)]
time150 = [2.43575,2.47981,2.51275,2.72231,3.409,3.92847,5.68224,22.4496]
time_sigma150 =[12.7222,13.0423,12.9583,15.0331,18.4065,18.3952,22.9263,67.3271]

prob_success100 = [0.321711,0.317475,0.3125,0.30843,0.294564,0.275741,0.250585,0.172958]
prob_sigma100 = [0.467133,0.465496,0.463514,0.461847,0.455848,0.446889,0.433351,0.378212]
mean100 = [198535.,197124.,179085.,177517.,104698.,35567.,31121.6,9651.81]
mean_sigma100 = [3.93484*10^6,6.08296*10^6,3.69632*10^6,6.68483*10^6,3.79012*10^6,351135.,1.0642*10^6,2443.94]
number_of_sims100 = [sqrt(634443),sqrt(150188),sqrt(147523),sqrt(140800),sqrt(148501),sqrt(152400),sqrt(156390),sqrt(173221)]
time100 = [8.4226,9.49264,10.4432,11.5911,13.5737,14.048,17.1391,38.1711]
time_sigma100 =[38.7737,41.7508,44.3292,46.8041,50.5953,48.1733,50.771,83.0854]

prob_success50 = [0.179698,0.17358,0.170547,0.164953,0.154619,0.142073,0.126716,0.0820594]
prob_sigma50 = [0.383937,0.37875,0.376114,0.37114,0.361543,0.349127,0.332656,0.274456]
mean50 = [143826.,127601.,112551.,118862.,73341.7,33654.7,17877.6,9187.57]
mean_sigma50 = [2.15731*10^6,2.31411*10^6,1.63087*10^6,2.63169*10^6,1.08651*10^6,659450.,176773.,2301.57]
number_of_sims50 = [sqrt(134175),sqrt(100029),sqrt(103086),sqrt(103381),sqrt(102730),sqrt(105664),sqrt(108487),sqrt(121473)]
time50 = [12.9067,13.7628,14.2784,15.0591,16.3064,16.512,17.8204,28.2929]
time_sigma50 =[49.884,51.9084,52.5574,54.2098,55.9658,54.2856,54.3863,71.7908]

prob_success0 = [0.0609653,0.0580386,0.0579275,0.0555649,0.0505348,0.0465496,0.0416369,0.0262047]
prob_sigma0 = [0.239267,0.233818,0.233607,0.22908,0.219047,0.210673,0.199759,0.159744]
mean0 = [124420.,104145.,94199.3,102888.,57298.3,20862.6,19442.7,9534.08]
mean_sigma0 = [3.38188*10^6,2.07578*10^6,1.06524*10^6,1.47331*10^6,1.12906*10^6,149171.,352002.,2386.96]
number_of_sims0 = [sqrt(1342140),sqrt(104999),sqrt(152501),sqrt(148007),sqrt(105175),sqrt(134244),sqrt(137114),sqrt(104981)]
time0 = [8.35639,8.78197,9.04973,9.23944,9.65523,9.46847,9.74487,13.3427]
time_sigma0 =[39.5505,40.7133,41.3733,41.7399,42.4668,40.7554,40.3345,49.0281]


prob_success = [prob_success200 prob_success150 prob_success100 prob_success50 prob_success0]
prob_sigma = [prob_sigma200/number_of_sims200 prob_sigma150/number_of_sims150 prob_sigma100/number_of_sims100 prob_sigma50/number_of_sims50 prob_sigma0/number_of_sims0]

mean = [mean200 mean150 mean100 mean50 mean0]
mean_sigma = [mean_sigma200/number_of_sims200 mean_sigma150/number_of_sims150 mean_sigma100/number_of_sims100 mean_sigma50/number_of_sims50 mean_sigma0/number_of_sims0]

time = [time200 time150 time100 time50 time0]
time_sigma = [time_sigma200/number_of_sims200 time_sigma150/number_of_sims150 time_sigma100/number_of_sims100 time_sigma50/number_of_sims50 time_sigma0/number_of_sims0]

plot()
plot!(delays,prob_success,yerror=prob_sigma,ribbon=prob_sigma,fillalpha= 0.2,grid=true,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p1 = plot!(delays,prob_success,grid=true,color = "Black",alpha=0.7,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:line,xlims = (-2,102),font = "Ariel") 

plot()
plot!(delays,mean,yerror=mean_sigma,grid=true,ribbon=mean_sigma,fillalpha= 0.2,yaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p3 = plot!(delays,mean,grid=true,color = "Black",alpha=0.7,yaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:line,xlims = (-2,102),font = "Ariel")

plot()
plot!(delays,prob_success,yerror=prob_sigma,grid=true,ribbon=prob_sigma,fillalpha= 0.2,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p2 = plot!(delays,prob_success,grid=true,color = "Black",alpha=0.7,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot(delays,mean,yerror=mean_sigma,grid=true,ribbon=mean_sigma,fillalpha= 0.2,yaxis=:log,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p4 = plot!(delays,mean,color = "Black",alpha=0.7,grid=true,yaxis=:log,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot(delays,time,yerror=time_sigma,grid=true,ribbon=time_sigma,fillalpha= 0.2,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p5 = plot!(delays,time,color = "Black",alpha=0.7,grid=true,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:line,xlims = (-2,102),font = "Ariel")

plot()
plot!(delays,time,yerror=time_sigma,grid=true,ribbon=time_sigma,fillalpha= 0.2,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p6 = plot!(delays,time,color = "Black",alpha=0.7,grid=true,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot(p1, p2, p3, p4, p5, p6, layout = l,size =(1600,1800),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25))

In [ ]:
#plot for β₃
l = @layout [a b; c d; e f]
delays = [0,0.5,1,2,10,50,100,365]

prob_success10n4 = [0.580016,0.576499,0.578837,0.578279,0.580816,0.576478,0.579891,0.577501]
prob_sigma10n4 = [0.493557,0.494116,0.493748,0.493837,0.493428,0.494119,0.493578,0.493959]
mean10n4 = [1.53191*10^6,1.34259*10^6,1.38166*10^6,1.28288*10^6,759544,340114,217968,20408.3]
mean_sigma10n4 = [1.13528*10^7,7.40251*10^6,1.08067*10^7,9.9858*10^6,4.75756*10^6,4.85713*10^6,3.0557*10^6,527350]
number_of_sims10n4 = [sqrt(256731),sqrt(106538),sqrt(105705),sqrt(105501),sqrt(105617),sqrt(114105),sqrt(121666),sqrt(143418)]
time10n4 = [1.64546,1.67286,1.62925,1.68716,1.68003,1.61089,1.71913,1.86218]
time_sigma10n4 =[7.9846,8.53064,8.04551,8.79182,8.63927,7.44698,8.66009,10.9508]

prob_success4t10n5 = [0.321711,0.317475,0.3125,0.30843,0.294564,0.275741,0.250585,0.172958]
prob_sigma4t10n5 = [0.467133,0.465496,0.463514,0.461847,0.455848,0.446889,0.433351,0.378212]
mean4t10n5 = [198535.,197124.,179085.,177517.,104698.,35567.,31121.6,9651.81]
mean_sigma4t10n5 = [3.93484*10^6,6.08296*10^6,3.69632*10^6,6.68483*10^6,3.79012*10^6,351135.,1.0642*10^6,2443.94]
number_of_sims4t10n5 = [sqrt(634443),sqrt(150188),sqrt(147523),sqrt(140800),sqrt(148501),sqrt(152400),sqrt(156390),sqrt(173221)]
time4t10n5 = [8.4226,9.49264,10.4432,11.5911,13.5737,14.048,17.1391,38.1711]
time_sigma4t10n5 =[38.7737,41.7508,44.3292,46.8041,50.5953,48.1733,50.771,83.0854]

prob_success10n5 = [0.0626245,0.0630946,0.0618317,0.0630908,0.0603281,0.0597871,0.0571256,0.0498191]
prob_sigma10n5 = [0.242287,0.243134,0.240851,0.243127,0.238095,0.237093,0.232084,0.217572]
mean10n5 = [9489.89,9626.14,9471.38,9471.57,9255.95,9130.65,9068.35,8862.56]
mean_sigma10n5 = [4057.02,5268.35,2538.75,4847.29,2827.06,2398.44,2381.86,2249.34]
number_of_sims10n5 = [sqrt(127490),sqrt(100595),sqrt(100434),sqrt(101140),sqrt(102357),sqrt(104454),sqrt(106257),sqrt(119934)]
time10n5 = [19.3006,19.2903,19.2524,19.4266,19.2971,19.2389,19.1301,21.0142]
time_sigma10n5 =[60.0717,60.0054,60.2217,60.0074,59.8518,59.5921,58.7898,61.7948]


prob_success = [prob_success10n4 prob_success4t10n5 prob_success10n5]
prob_sigma = [prob_sigma10n4/number_of_sims10n4 prob_sigma4t10n5/number_of_sims4t10n5 prob_sigma10n5/number_of_sims10n5]

mean = [mean10n4 mean4t10n5 mean10n5]
mean_sigma = [mean_sigma10n4/number_of_sims10n4 mean_sigma4t10n5/number_of_sims4t10n5 mean_sigma10n5/number_of_sims10n5]

time = [time10n4 time4t10n5 time10n5]
time_sigma = [time_sigma10n4/number_of_sims10n4 time_sigma4t10n5/number_of_sims4t10n5 time_sigma10n5/number_of_sims10n5]

plot()
plot!(delays,prob_success,yerror=prob_sigma,ribbon=prob_sigma,fillalpha= 0.2,grid=true,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel") 
p1 = plot!(delays,prob_success,color = "Black",alpha=0.7,grid=true,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:line,xlims = (-2,102),font = "Ariel") 

plot()
plot!(delays,mean,yerror=mean_sigma,grid=true,ribbon=mean_sigma,fillalpha= 0.2,yaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p3 = plot!(delays,mean,color = "Black",alpha=0.7,grid=true,yaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:line,xlims = (-2,102),font = "Ariel")

plot()
plot!(delays,prob_success,yerror=prob_sigma,grid=true,ribbon=prob_sigma,fillalpha= 0.2,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p2 = plot!(delays,prob_success,color = "Black",alpha=0.7,grid=true,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot!(delays,mean,yerror=mean_sigma,grid=true,ribbon=mean_sigma,fillalpha= 0.2,yaxis=:log,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p4 = plot!(delays,mean,color = "Black",alpha=0.7,grid=true,yaxis=:log,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot!(delays,time,yerror=time_sigma,grid=true,ribbon=time_sigma,fillalpha= 0.2,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p5 = plot!(delays,time,color = "Black",alpha=0.7,grid=true,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:line,xlims = (-2,102),font = "Ariel")

plot()
plot!(delays,time,yerror=time_sigma,grid=true,ribbon=time_sigma,fillalpha= 0.2,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p6 = plot!(delays,time,color = "Black",alpha=0.7,grid=true,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot(p1, p2, p3, p4, p5, p6, layout = l,size =(1600,1800),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25))

In [ ]:
#plot for β₄
l = @layout [a b; c d; e f]
delays = [0,0.5,1,2,10,50,100,365]

prob_success5t10n4 = [0.321253,0.318637,0.313757,0.307416,0.295281,0.278051,0.254158,0.172117]
prob_sigma5t10n4 = [0.466959,0.46595,0.464021,0.461425,0.456171,0.448041,0.435389,0.377483]
mean5t10n4 = [172414,204083,170563,167353,107548,34361.6,28335.5,9377.94]
mean_sigma5t10n4 = [3.2732*10^6,6.77102*10^6,2.42371*10^6,2.44056*10^6,2.74728*10^6,323682,609004,2387.1]
number_of_sims5t10n4 = [sqrt(217738),sqrt(124518),sqrt(125441),sqrt(124395),sqrt(125589),sqrt(127980),sqrt(129805),sqrt(146017)]
time5t10n4 = [8.46267,9.37942,10.072,11.3366,13.8726,14.1926,17.2497,38.5379]
time_sigma5t10n4 =[39.0201,41.382,43.2191,46.4694,51.5219,48.7865,50.8526,83.7162]

prob_success10n4 = [0.321711,0.317475,0.3125,0.30843,0.294564,0.275741,0.250585,0.172958]
prob_sigma10n4 = [0.467133,0.465496,0.463514,0.461847,0.455848,0.446889,0.433351,0.378212]
mean10n4 = [198535.,197124.,179085.,177517.,104698.,35567.,31121.6,9651.81]
mean_sigma10n4 = [3.93484*10^6,6.08296*10^6,3.69632*10^6,6.68483*10^6,3.79012*10^6,351135.,1.0642*10^6,2443.94]
number_of_sims10n4 = [sqrt(634443),sqrt(150188),sqrt(147523),sqrt(140800),sqrt(148501),sqrt(152400),sqrt(156390),sqrt(173221)]
time10n4 = [8.4226,9.49264,10.4432,11.5911,13.5737,14.048,17.1391,38.1711]
time_sigma10n4 =[38.7737,41.7508,44.3292,46.8041,50.5953,48.1733,50.771,83.0854]

prob_success5t10n5 = [0.320807,0.317287,0.314393,0.309786,0.294581,0.276185,0.249058,0.171064]
prob_sigma5t10n5 = [0.466787,0.465423,0.464276,0.462408,0.455856,0.447112,0.43247,0.376566]
mean5t10n5 = [214865,176308,189249,152551,133333,37374.8,21412,9684.41]
mean_sigma5t10n5 = [5.89724*10^6,2.21887*10^6,3.03558*10^6,1.87603*10^6,3.96456*10^6,548492,194234,2452.99]
number_of_sims5t10n5 = [sqrt(431599),sqrt(102144),sqrt(101599),sqrt(100876),sqrt(101439),sqrt(102663),sqrt(106449),sqrt(118903)]
time5t10n5 = [8.42075,9.23028,10.2403,11.52,13.9203,14.1769,17.0167,38.684]
time_sigma5t10n5 =[38.8021,40.6315,43.6771,46.7931,51.5811,48.3473,49.916,84.0469]


prob_success = [prob_success5t10n4 prob_success10n4 prob_success5t10n5]
prob_sigma = [prob_sigma5t10n4/number_of_sims5t10n4 prob_sigma10n4/number_of_sims10n4 prob_sigma5t10n5/number_of_sims5t10n5]

mean = [mean5t10n4 mean10n4 mean5t10n5]
mean_sigma = [mean_sigma5t10n4/number_of_sims5t10n4 mean_sigma10n4/number_of_sims10n4 mean_sigma5t10n5/number_of_sims5t10n5]

time = [time5t10n4 time10n4 time5t10n5]
time_sigma = [time_sigma5t10n4/number_of_sims5t10n4 time_sigma10n4/number_of_sims10n4 time_sigma5t10n5/number_of_sims5t10n5]

plot()
plot!(delays,prob_success,yerror=prob_sigma,ribbon=prob_sigma,fillalpha= 0.2,grid=true,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p1 = plot!(delays,prob_success,color = "Black",alpha=0.7,grid=true,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:line,xlims = (-2,102),font = "Ariel")

plot()
plot!(delays,mean,yerror=mean_sigma,grid=true,ribbon=mean_sigma,fillalpha= 0.2,yaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p3 = plot!(delays,mean,color = "Black",alpha=0.7,grid=true,yaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:line,xlims = (-2,102),font = "Ariel")

plot()
plot!(delays,prob_success,yerror=prob_sigma,grid=true,ribbon=prob_sigma,fillalpha= 0.2,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p2 = plot!(delays,prob_success,color = "Black",alpha=0.7,grid=true,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Probability of establishment", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot!(delays,mean,yerror=mean_sigma,grid=true,ribbon=mean_sigma,fillalpha= 0.2,yaxis=:log,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p4 = plot!(delays,mean,color = "Black",alpha=0.7,grid=true,yaxis=:log,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean size", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot!(delays,time,yerror=time_sigma,grid=true,ribbon=time_sigma,fillalpha= 0.2,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:scatter,xlims = (-2,102),markersize = 12,font = "Ariel")
p5 = plot!(delays,time,color = "Black",alpha=0.7,grid=true,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:line,xlims = (-2,102),font = "Ariel")

plot()
plot!(delays,time,yerror=time_sigma,grid=true,ribbon=time_sigma,fillalpha= 0.2,xaxis=:log,palette=:tab10,align="center",alpha=0.7,label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:scatter,xlims = (0.4,420),markersize = 12,font = "Ariel")
p6 = plot!(delays,time,color = "Black",alpha=0.7,grid=true,xaxis=:log,align="center",label = "",xlabel = "Delay τ (days)",ylabel = "Mean time to extinction", seriestype =:line,xlims = (0.4,420),font = "Ariel")

plot()
plot(p1, p2, p3, p4, p5, p6, layout = l,size =(1600,1800),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25))

# Figure S5

In [ ]:
using Plots

In [ ]:
#time to extinction plot
l = @layout [a b]
delays = [0,0.5,1,2,10,50,100,365]
alpha1 = [0,50,100,150,200]
times0 = [8.35639,12.9067,8.4226,2.43575,1.8359]
times365 = [13.3427,28.2929,38.1711,22.4496,2.14636]

logocolors = Colors.JULIA_LOGO_COLORS

p1 = plot(alpha1,times0,grid=true,color=logocolors.purple,align="center",title = "τ = 0",xlabel = "MDSCs circulating rate α₂",ylabel = "Mean time (days)",legend = false,seriestype =:scatter,markersize = 25,font = "Ariel",xlims = (-10,210),ylims = (0,45))
p2 = plot(alpha1,times365,grid=true,color=logocolors.purple,align="center",title = "τ = 365",xlabel = "MDSCs circulating rate α₂",ylabel = "Mean time (days)",legend = false,seriestype =:scatter,markersize = 25,font = "Ariel",xlims = (-10,210),ylims = (0,45))
plot(p1,p2,layout = l,size =(1600,600),xtickfont=font(25),ytickfont=font(25),guidefont=font(25),legendfont=font(25),titlefont=font(25))

# Figure S9

In [ ]:
using Catalyst
using DifferentialEquations
using ModelingToolkit
using Plots
using Statistics
using ParameterizedFunctions
using DelimitedFiles
using CSV
using DataFrames
import Random
logocolors = Colors.JULIA_LOGO_COLORS

In [ ]:
# model definition 
system = @reaction_network begin
  α₁*T*log(η/T),  0 --> T
  α₂, 0 --> MDSC
  β₁*NK, T --> 0
  β₂*CTL, T --> 0
  ζ₁, T --> 0
  α₃*T/(γ₁+(T)), 0 --> MDSC
  ζ₂, MDSC --> 0
  α₄, 0 --> NK
  α₅*(T^2)/(γ₂+(T^2)), 0 --> NK
  β₃*MDSC, NK --> 0
  ζ₃, NK --> 0
  α₆*T*NK, 0 --> CTL
  α₇*(T^2)/(γ₃+(T^2)), 0 --> CTL
  β₄*MDSC, CTL --> 0
  ζ₄, CTL --> 0
end α₂ α₃ ζ₂ α₁ η β₁ β₂ ζ₁ α₄ α₅ β₃ ζ₃ α₆ α₇ β₄ ζ₄ γ₁ γ₂ γ₃

In [ ]:
fullsystem = convert(ODESystem, system)

In [ ]:
#set model parameters
α₂ = 10.0*(10.0^1.0) #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (α₂,α₃,ζ₂,α₁,η,β₁,β₂,ζ₁,α₄,α₅,β₃,ζ₃,α₆,α₇,β₄,ζ₄,γ₁,γ₂,γ₃)
tspan = (0.0,365.0)
u0 = [1.0;α₂/ζ₂;ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃);0.0]

In [ ]:
# deterministic simulation of system
op    = ODEProblem(system, u0, tspan, p)
sol   = solve(op, Tsit5()) 
plot2 = plot(sol, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

display(plot2)
sol

In [ ]:
#define conditions for simulation to end
function condition(u,t,integrator) 
  u[1] <= 0 
end
floor_event = DiscreteCallback(condition, terminate!)

## Panel A

In [ ]:
# stochastic simulation of system - successful tumor
Random.seed!(1240)

dprob = DiscreteProblem(system, u0, tspan, p)
jprob = JumpProblem(system, dprob, Direct(), save_positions=(false,false)) 
jsol = solve(jprob, SSAStepper(), saveat=1.0, callback=floor_event)
plot1 = plot(jsol, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

display(plot1)

jsol

## Panel B

In [ ]:
# stochastic simulation of system - unsuccessful tumor

Random.seed!(1259)

dprob = DiscreteProblem(system, u0, tspan, p)
jprob = JumpProblem(system, dprob, Direct(), save_positions=(false,false)) 
jsol = solve(jprob, SSAStepper(), saveat=1.0, callback=floor_event)
plot1 = plot(jsol, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", xlims = (0.0,10.0), ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

display(plot1)

jsol

In [ ]:
#parameters
α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells

1 / (α₁*1.0*log(η/1.0) / (β₁*228758.1699346405))

In [ ]:
df = DataFrame(CSV.File("gillespie.csv"))
list = Matrix(df)
list2 = zeros(length(list))
for i = 1:length(list)
    list2[i] = mean(list[1:i])
end
vector = LinRange(1, length(list2), length(list2))

## Panel C

In [ ]:
#plot
plot()

success = 1 - (1 / (α₁*log(η) / (β₁*(ζ₂*α₄/(α₂*β₃+ζ₂*ζ₃))-ζ₁)))
success_vector = zeros(length(vector)) .+ success

plot!(vector, list2, xlims = (8, 25000), ylims = (0.2,0.8), xaxis = :log, lw = 4, size =(1600,600), font = "Ariel", color = logocolors.purple, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Number of simulations",xguidefontsize=25,ylabel="Rolling mean",yguidefontsize=25,title = "Probability of establishment",titlefontsize=25)
plot!(vector, success_vector, lw = 2, linestyle = :dash, size =(1600,600), font = "Ariel", color = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Number of simulations",xguidefontsize=25,ylabel="Rolling mean",yguidefontsize=25,title = "Probability of establishment",titlefontsize=25)


# Figures 6, S10, and S11

In [ ]:
using Turing, Distributions, DifferentialEquations
using MCMCChains, Plots, StatsPlots

import Random
Random.seed!(14);

logocolors = Colors.JULIA_LOGO_COLORS

In [ ]:
color1 = RGBA(113/255,46/255,103/255,1)
color2 = RGBA(38/255,117/255,146/255,1)
color3 = RGBA(95/255,177/255,42/255,1)
color4 = RGBA(199/255,200/255,0/255,1)
color5 = RGBA(255/255,121/255,23/255,1)
color6 = RGBA(226/255,58/255,52/255,1)

## Data

In [ ]:
#tumor data
time_1 = [0.0,39.0,82.0,124.0,165.0,249.0,277.0,292.0,334.0]

tumor_1 = [24565000000.0,29160000000.0,20480000000.0,3645000000.0,2560000000.0,3645000000.0,1715000000.0,1715000000.0,1715000000.0]
for i = 2:convert(Int64, length(tumor_1))
    tumor_1[i] = (tumor_1[i]-tumor_1[1])./tumor_1[1]
end
tumor_1[1] = 0.0


time_2 = [0.0,43.0,84.0,127.0,169.0,211.0,253.0,295.0,344.0]

tumor_2 = [5.5296E+11,1.9652E+11,1.9652E+11,1.9652E+11,1.0976E+11,78125000000.0,78125000000.0,87880000000.0,1.0976E+11]
for i = 2:convert(Int64, length(tumor_2))
    tumor_2[i] = (tumor_2[i]-tumor_2[1])./tumor_2[1]
end
tumor_2[1] = 0.0


time_3 = [0.0,41.0,83.0,125.0,167.0,210.0,251.0,293.0,334.0,442.0,377.0]

tumor_3 = [1.37313E+12,1.37313E+12,4.8668E+11,4.8668E+11,3.2E+11,3.44605E+11,2.7436E+11,2.7436E+11,3.97535E+11,4.55625E+11,4.2592E+11]
for i = 2:convert(Int64, length(tumor_3))
    tumor_3[i] = (tumor_3[i]-tumor_3[1])./tumor_3[1]
end
tumor_3[1] = 0.0

time_4 = [0.0,42.0,84.0,127.0,169.0,211.0,253.0]

tumor_4 = [3.2E+11,2.96595E+11,2.14375E+11,2.14375E+11,3.2E+11,3.2E+11,3.44605E+11]
for i = 2:convert(Int64, length(tumor_4))
    tumor_4[i] = (tumor_4[i]-tumor_4[1])./tumor_4[1]
end
tumor_4[1] = 0.0

time_5 = [0.0,41.0,83.0,125.0,167.0,210.0,251.0,293.0,335.0,393.0,461.0]

tumor_5 = [1.0976E+11,1.0976E+11,1.35E+11,1.79685E+11,1.79685E+11,2.7436E+11,3.44605E+11,3.7044E+11,4.8668E+11,5.5296E+11,5.5296E+11]
for i = 2:convert(Int64, length(tumor_5))
    tumor_5[i] = (tumor_5[i]-tumor_5[1])./tumor_5[1]
end
tumor_5[1] = 0.0

time_6 = [0.0,55.0,90.0,118.0,167.0]

tumor_6 = [3.97535E+11,5.19115E+11,6.25E+11,6.25E+11,1.37313E+12]
for i = 2:convert(Int64, length(tumor_6))
    tumor_6[i] = (tumor_6[i]-tumor_6[1])./tumor_6[1]
end
tumor_6[1] = 0.0

In [ ]:
#plot tumor data relative change
scatter()
scatter!(time_1, tumor_1, markersize = 10, size =(1600,600), font = "Ariel", color_palette = palette([color1, color1],2), legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_1, tumor_1, size =(1600,600), font = "Ariel", color = color1, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

scatter!(time_2, tumor_2, markersize = 10, size =(1600,600), font = "Ariel", color_palette = palette([color2, color2],2), legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_2, tumor_2, size =(1600,600), font = "Ariel", color = color2, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

scatter!(time_3, tumor_3, markersize = 10, size =(1600,600), font = "Ariel", color_palette = palette([color3, color3],2), legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_3, tumor_3, size =(1600,600), font = "Ariel", color = color3, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

scatter!(time_4, tumor_4, markersize = 10, size =(1600,600), font = "Ariel", color_palette = palette([color4, color4],2), legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_4, tumor_4, size =(1600,600), font = "Ariel", color = color4, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

scatter!(time_5, tumor_5, markersize = 10, size =(1600,600), font = "Ariel", color_palette = palette([color5, color5],2), legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_5, tumor_5, size =(1600,600), font = "Ariel", color = color5, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

scatter!(time_6, tumor_6, markersize = 10, size =(1600,600), font = "Ariel", color_palette = palette([color6, color6],2), legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_6, tumor_6, size =(1600,600), font = "Ariel", color = color6, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

In [ ]:
#remove first data point from each tumor (baseline)
popfirst!(tumor_1)
popfirst!(time_1)
popfirst!(tumor_2)
popfirst!(time_2)
popfirst!(tumor_3)
popfirst!(time_3)
popfirst!(tumor_4)
popfirst!(time_4)
popfirst!(tumor_5)
popfirst!(time_5)
popfirst!(tumor_6)
popfirst!(time_6)

In [ ]:
tumor_1

In [ ]:
tumor_2

In [ ]:
tumor_3

In [ ]:
tumor_4

In [ ]:
tumor_5

In [ ]:
tumor_6

## Model

In [ ]:
#define model without delay
function ode_system(du,u,p,t)
    T,MDSC,NK,CTL = u
    β₃, α₆, α₁ = p 
   
# Cancer cells (tumor)
    du[1] = dT = α₁*T*log(max(η/T,1)) - β₁*T*NK - β₂*T*CTL - ζ₁*T
 # MDSCs 
    du[2] = dMDSC = α₂ + α₃*T/(γ₁+(T)) - ζ₂*MDSC 
 # NK cells
    du[3] = dNK = α₄ + α₅*(T^2)/(γ₂+(T^2)) - β₃*MDSC*NK - ζ₃*NK 
 # CTL cells
    du[4] = dCTL = α₆*T*NK + α₇*(T^2)/(γ₃+(T^2)) - β₄*MDSC*CTL - ζ₄*CTL
end

In [ ]:
#set numerical solution algorithm
alg = Tsit5()

In [ ]:
#define condition for simulation to end
function condition(u,t,integrator) 
  u[1] < 0.0 || u[2] < 0.0 || u[3] < 0.0 || u[4] < 0.0
end

floor_event = DiscreteCallback(condition, terminate!)

In [ ]:
#set model parameters
α₂ = 10.0*(10.0^1.0) #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

α₁ = 10.0^(-1.0) #tumor growth rate
η = 10.0^7.0 #tumor maximum size 
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
β₃ = 4*(10.0^(-5.0)) #NK cells inactivation rate by MDSCs
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #(10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (β₃, α₆, α₁) 

tspan = (0.0,600.0)
u0 = [8395.368084462818, 804.0710624094437, 197565.74910954377, 1654.4182572290226] 

prob1 = ODEProblem(ode_system,u0,tspan,p)
sol = solve(prob1,alg,saveat=0.1,callback=floor_event)
plot(sol, lw = 4, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

In [ ]:
sol

## Fitting

In [ ]:
Threads.nthreads()

In [ ]:
#set model parameters
η = 10.0^(5.0)
η

In [ ]:
time_to_fit1 = time_1
tumor_to_fit1 = tumor_1
tumor_curve_fit1 = zeros(length(tumor_to_fit1))
for i = 1:convert(Int64, length(tumor_to_fit1))
    tumor_curve_fit1[i] = tumor_to_fit1[i].*u0[1]+u0[1]
end

In [ ]:
#fit tumor 1
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    
    β₃ ~ truncated(Normal(4.0*10.0^(-5.0),10.0^(-4.0)),0.0,10.0^(-2.0))
    α₆ ~ truncated(Normal(1.1*(10.0^(-7.0)),(10.0^(-6.0))),0.0,10.0^(-2.0))
    α₁ ~ truncated(Normal(10.0^(-1.0),3.0*10.0^(-1.0)),0.0,1.0)
    σ ~ InverseGamma(2, 3) 


    p = [β₃, α₆, α₁] 
    prob = remake(prob1, p=p)
    predicted = solve(prob,Tsit5(),saveat=time_to_fit1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
    

    for i = 1:min(length(predicted),length(tumor_curve_fit1))
        data[i] ~ Normal(log.(predicted[i][1].+1), σ) 
    end

end

model = fitlv(log.(tumor_curve_fit1.+1), prob1) 

@time chain1 = sample(model, NUTS(0.65), MCMCThreads(), 2000, 4, progress=true)

In [ ]:
time_to_fit2 = time_2
tumor_to_fit2 = tumor_2
tumor_curve_fit2 = zeros(length(tumor_to_fit2))
for i = 1:convert(Int64, length(tumor_to_fit2))
    tumor_curve_fit2[i] = tumor_to_fit2[i].*u0[1]+u0[1]
end

In [ ]:
#fit tumor 2
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    
    β₃ ~ truncated(Normal(4.0*10.0^(-5.0),10.0^(-4.0)),0.0,10.0^(-2.0))
    α₆ ~ truncated(Normal(1.1*(10.0^(-7.0)),(10.0^(-6.0))),0.0,10.0^(-2.0))
    α₁ ~ truncated(Normal(10.0^(-1.0),3.0*10.0^(-1.0)),0.0,1.0)
    σ ~ InverseGamma(2, 3) 


    p = [β₃, α₆, α₁] 
    prob = remake(prob1, p=p)
    predicted = solve(prob,Tsit5(),saveat=time_to_fit2,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
    

    for i = 1:min(length(predicted),length(tumor_curve_fit2)) 
        data[i] ~ Normal(log.(predicted[i][1].+1), σ)
    end

end

model = fitlv(log.(tumor_curve_fit2.+1), prob1) 

@time chain2 = sample(model, NUTS(0.65), MCMCThreads(), 2000, 4, progress=true)

In [ ]:
time_to_fit3 = time_3
tumor_to_fit3 = tumor_3
tumor_curve_fit3 = zeros(length(tumor_to_fit3))
for i = 1:convert(Int64, length(tumor_to_fit3))
    tumor_curve_fit3[i] = tumor_to_fit3[i].*u0[1]+u0[1]
end

In [ ]:
#fit tumor 3
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    
    β₃ ~ truncated(Normal(4.0*10.0^(-5.0),10.0^(-4.0)),0.0,10.0^(-2.0))
    α₆ ~ truncated(Normal(1.1*(10.0^(-7.0)),(10.0^(-6.0))),0.0,10.0^(-2.0))
    α₁ ~ truncated(Normal(10.0^(-1.0),3.0*10.0^(-1.0)),0.0,1.0)
    σ ~ InverseGamma(2, 3) 


    p = [β₃, α₆, α₁] 
    prob = remake(prob1, p=p)
    predicted = solve(prob,Tsit5(),saveat=time_to_fit3,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    

    for i = 1:min(length(predicted),length(tumor_curve_fit3))
        data[i] ~ Normal(log.(predicted[i][1].+1), σ)
    end

end

model = fitlv(log.(tumor_curve_fit3.+1), prob1) 

@time chain3 = sample(model, NUTS(0.65), MCMCThreads(), 2000, 4, progress=true)

In [ ]:
time_to_fit4 = time_4
tumor_to_fit4 = tumor_4
tumor_curve_fit4 = zeros(length(tumor_to_fit4))
for i = 1:convert(Int64, length(tumor_to_fit4))
    tumor_curve_fit4[i] = tumor_to_fit4[i].*u0[1]+u0[1]
end

In [ ]:
#fit tumor 4
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    
    β₃ ~ truncated(Normal(4.0*10.0^(-5.0),10.0^(-4.0)),0.0,10.0^(-2.0))
    α₆ ~ truncated(Normal(1.1*(10.0^(-7.0)),(10.0^(-6.0))),0.0,10.0^(-2.0))
    α₁ ~ truncated(Normal(10.0^(-1.0),3.0*10.0^(-1.0)),0.0,1.0)
    σ ~ InverseGamma(2, 3) 


    p = [β₃, α₆, α₁] 
    prob = remake(prob1, p=p)
    predicted = solve(prob,Tsit5(),saveat=time_to_fit4,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    

    for i = 1:min(length(predicted),length(tumor_curve_fit4))
        data[i] ~ Normal(log.(predicted[i][1].+1), σ) 
    end

end

model = fitlv(log.(tumor_curve_fit4.+1), prob1) 

@time chain4 = sample(model, NUTS(0.65), MCMCThreads(), 2000, 4, progress=true)

In [ ]:
time_to_fit5 = time_5
tumor_to_fit5 = tumor_5
tumor_curve_fit5 = zeros(length(tumor_to_fit5))
for i = 1:convert(Int64, length(tumor_to_fit5))
    tumor_curve_fit5[i] = tumor_to_fit5[i].*u0[1]+u0[1]
end


In [ ]:
#fit tumor 5
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    
    β₃ ~ truncated(Normal(4.0*10.0^(-5.0),10.0^(-4.0)),0.0,10.0^(-2.0))
    α₆ ~ truncated(Normal(1.1*(10.0^(-7.0)),(10.0^(-6.0))),0.0,10.0^(-2.0))
    α₁ ~ truncated(Normal(10.0^(-1.0),3.0*10.0^(-1.0)),0.0,1.0)
    σ ~ InverseGamma(2, 3) 


    p = [β₃, α₆, α₁] 
    prob = remake(prob1, p=p)
    predicted = solve(prob,Tsit5(),saveat=time_to_fit5,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
    

    for i = 1:min(length(predicted),length(tumor_curve_fit5)) 
        data[i] ~ Normal(log.(predicted[i][1].+1), σ) 
    end

end

model = fitlv(log.(tumor_curve_fit5.+1), prob1) 

@time chain5 = sample(model, NUTS(0.65), MCMCThreads(), 2000, 4, progress=true)

In [ ]:
time_to_fit6 = time_6
tumor_to_fit6 = tumor_6
tumor_curve_fit6 = zeros(length(tumor_to_fit6))
for i = 1:convert(Int64, length(tumor_to_fit6))
    tumor_curve_fit6[i] = tumor_to_fit6[i].*u0[1]+u0[1]
end

In [ ]:
#fit tumor 6
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    
    β₃ ~ truncated(Normal(4.0*10.0^(-5.0),10.0^(-4.0)),0.0,10.0^(-2.0))
    α₆ ~ truncated(Normal(1.1*(10.0^(-7.0)),(10.0^(-6.0))),0.0,10.0^(-2.0))
    α₁ ~ truncated(Normal(10.0^(-1.0),3.0*10.0^(-1.0)),0.0,1.0)
    σ ~ InverseGamma(2, 3) 


    p = [β₃, α₆, α₁] 
    prob = remake(prob1, p=p)
    predicted = solve(prob,Tsit5(),saveat=time_to_fit6,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
    

    for i = 1:min(length(predicted),length(tumor_curve_fit6)) 
        data[i] ~ Normal(log.(predicted[i][1].+1), σ) 
    end

end

model = fitlv(log.(tumor_curve_fit6.+1), prob1) 

@time chain6 = sample(model, NUTS(0.65), MCMCThreads(), 2000, 4, progress=true)

In [ ]:
chain_array1 = Array(chain1)

In [ ]:
chain_array2 = Array(chain2)

In [ ]:
chain_array3 = Array(chain3)

In [ ]:
chain_array4 = Array(chain4)

In [ ]:
chain_array5 = Array(chain5)

In [ ]:
chain_array6 = Array(chain6)

In [ ]:
#plot joint posterior
scatter()
scatter!(chain_array1[:,1], chain_array1[:,3], markercolor = palette([color1, color1],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,1), xlab = "β₃", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array2[:,1], chain_array2[:,3], markercolor = palette([color2, color2],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,1), xlab = "β₃", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array3[:,1], chain_array3[:,3], markercolor = palette([color3, color3],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,1), xlab = "β₃", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array4[:,1], chain_array4[:,3], markercolor = palette([color4, color4],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,1), xlab = "β₃", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array5[:,1], chain_array5[:,3], markercolor = palette([color5, color5],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,1), xlab = "β₃", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array6[:,1], chain_array6[:,3], markercolor = palette([color6, color6],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,1), xlab = "β₃", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)

In [ ]:
#plot joint posterior
scatter()
scatter!(chain_array1[:,1], chain_array1[:,2], markercolor = palette([color1, color1],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,5*10^(-6)), xlab = "β₃", ylab = "α₆", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array2[:,1], chain_array2[:,2], markercolor = palette([color2, color2],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,5*10^(-6)), xlab = "β₃", ylab = "α₆", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array3[:,1], chain_array3[:,2], markercolor = palette([color3, color3],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,5*10^(-6)), xlab = "β₃", ylab = "α₆", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array4[:,1], chain_array4[:,2], markercolor = palette([color4, color4],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,5*10^(-6)), xlab = "β₃", ylab = "α₆", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array5[:,1], chain_array5[:,2], markercolor = palette([color5, color5],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,5*10^(-6)), xlab = "β₃", ylab = "α₆", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array6[:,1], chain_array6[:,2], markercolor = palette([color6, color6],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0, 0.0005), ylims = (0,5*10^(-6)), xlab = "β₃", ylab = "α₆", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)

In [ ]:
#plot joint posterior
scatter()
scatter!(chain_array1[:,2], chain_array1[:,3], markercolor = palette([color1, color1],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0,5*10^(-6)), ylims = (0,1), xlab = "α₆", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array2[:,2], chain_array2[:,3], markercolor = palette([color2, color2],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0,5*10^(-6)), ylims = (0,1), xlab = "α₆", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array3[:,2], chain_array3[:,3], markercolor = palette([color3, color3],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0,5*10^(-6)), ylims = (0,1), xlab = "α₆", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array4[:,2], chain_array4[:,3], markercolor = palette([color4, color4],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0,5*10^(-6)), ylims = (0,1), xlab = "α₆", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array5[:,2], chain_array5[:,3], markercolor = palette([color5, color5],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0,5*10^(-6)), ylims = (0,1), xlab = "α₆", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)
scatter!(chain_array6[:,2], chain_array6[:,3], markercolor = palette([color6, color6],2), markerstrokewidth = 0, markeralpha = 0.05, xlims = (0,5*10^(-6)), ylims = (0,1), xlab = "α₆", ylab = "α₁", legend = :false, size = (1600/3,600), font = "Ariel", xtickfontsize=12,ytickfontsize=12,xguidefontsize=25,yguidefontsize=25,titlefontsize=25)

In [ ]:
#plot MCMCs
l = @layout [a b c d e f]
a = mixeddensity(chain1, size = (1600/2/2,600), font = "Ariel", plot_title="Tumor 1", xtickfontsize=5,ytickfontsize=8,xguidefontsize=12,yguidefontsize=12,titlefontsize=25)
b = mixeddensity(chain2, size = (1600/2/2,600), font = "Ariel", plot_title="Tumor 2", xtickfontsize=5,ytickfontsize=8,xguidefontsize=12,yguidefontsize=12,titlefontsize=25)
c = mixeddensity(chain3, size = (1600/2/2,600), font = "Ariel", plot_title="Tumor 3", xtickfontsize=5,ytickfontsize=8,xguidefontsize=12,yguidefontsize=12,titlefontsize=25)
d = mixeddensity(chain4, size = (1600/2/2,600), font = "Ariel", plot_title="Tumor 4", xtickfontsize=5,ytickfontsize=8,xguidefontsize=12,yguidefontsize=12,titlefontsize=25)
e = mixeddensity(chain5, size = (1600/2/2,600), font = "Ariel", plot_title="Tumor 5", xtickfontsize=5,ytickfontsize=8,xguidefontsize=12,yguidefontsize=12,titlefontsize=25)
f = mixeddensity(chain6, size = (1600/2/2,600), font = "Ariel", plot_title="Tumor 6", xtickfontsize=5,ytickfontsize=8,xguidefontsize=12,yguidefontsize=12,titlefontsize=25)
plot(a, b, c, d, e, f, layout = l, size =(1600,1200), plot_title ="", titlefontsize=25)

In [ ]:
#plot
plot()
α₂ = 10.0*(10.0^1.0) #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

η = 10.0^(5.0) #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (β₃, α₆, α₁) 

u0 = [8395.368084462818, 804.0710624094437, 197565.74910954377, 1654.4182572290226] #100

β₃ = mean(chain_array1[:,1]) 
α₆ = mean(chain_array1[:,2]) 
α₁ = mean(chain_array1[:,3]) 
tspan = (0.0,last(time_to_fit1)+20)
p = (β₃, α₆, α₁)
probplot = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth1 = solve(probplot,alg,saveat=0.1,callback=floor_event)

β₃ = mean(chain_array2[:,1]) 
α₆ = mean(chain_array2[:,2]) 
α₁ = mean(chain_array2[:,3]) 
tspan = (0.0,last(time_to_fit2)+20)
p = (β₃, α₆, α₁)
probplot = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth2 = solve(probplot,alg,saveat=0.1,callback=floor_event)

β₃ = mean(chain_array3[:,1]) 
α₆ = mean(chain_array3[:,2]) 
α₁ = mean(chain_array3[:,3]) 
tspan = (0.0,last(time_to_fit3)+20)
p = (β₃, α₆, α₁)
probplot = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth3 = solve(probplot,alg,saveat=0.1,callback=floor_event)

β₃ = mean(chain_array4[:,1]) 
α₆ = mean(chain_array4[:,2]) 
α₁ = mean(chain_array4[:,3]) 
tspan = (0.0,last(time_to_fit4)+20)
p = (β₃, α₆, α₁)
probplot = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth4 = solve(probplot,alg,saveat=0.1,callback=floor_event)

β₃ = mean(chain_array5[:,1]) 
α₆ = mean(chain_array5[:,2]) 
α₁ = mean(chain_array5[:,3])
tspan = (0.0,last(time_to_fit5)+20)
p = (β₃, α₆, α₁)
probplot = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth5 = solve(probplot,alg,saveat=0.1,callback=floor_event)

β₃ = mean(chain_array6[:,1]) 
α₆ = mean(chain_array6[:,2]) 
α₁ = mean(chain_array6[:,3]) 
tspan = (0.0,last(time_to_fit6)+20)
p = (β₃, α₆, α₁)
probplot = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth6 = solve(probplot,alg,saveat=0.1,callback=floor_event)

In [ ]:
#plot
plot()
α₂ = 10.0*(10.0^1.0) #MDSCs production rate
α₃ = (10.0^8.0) #MDSCs expansion coefficient 
ζ₂ = 0.2 #MDSCs death rate

η = 10.0^(5.0) #tumor maximum size
β₁ = 3.5*(10.0^(-6.0)) #tumor cells kill rate by NK cells
β₂ = 1.1*(10.0^(-7.0)) #tumor cells kill rate by T cells
ζ₁ = 0.0 #tumor cell death rate

α₄ = 1.4*(10.0^4.0) #NK cells production rate
α₅ = 2.5*(10.0^(-2.0)) #NK cells expansion coefficient
ζ₃ = 4.12*(10.0^(-2.0)) #NK cells death rate

α₆ = 1.1*(10.0^(-7.0)) #CTL stimulation as a result of tumor-NK cell interaction
α₇ = 10.0^(-1.0) #CTL expansion coefficient 
β₄ = (10.0^(-4.0)) #CTL inactivation rate by MDSCs
ζ₄ = 2.0*(10.0^(-2.0)) #CTL death rate

γ₁ = (10.0^(10.0)) #steepness coefficient of MDSC production curve
γ₂ = 2.02*(10.0^(7.0)) #steepness coefficient of NK production curve
γ₃ = 2.02*(10.0^(7.0)) #steepness coefficient of CTL production curve

p = (β₃, α₆, α₁) 

u0 = [8395.368084462818, 804.0710624094437, 197565.74910954377, 1654.4182572290226]

β₃ = median(chain_array1[:,1]) 
α₆ = median(chain_array1[:,2]) 
α₁ = median(chain_array1[:,3]) 
tspan = (0.0,last(time_to_fit1)+20)
p = (β₃, α₆, α₁)
probplot1 = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth1 = solve(probplot1,alg,saveat=0.1,callback=floor_event)

β₃ = median(chain_array2[:,1]) 
α₆ = median(chain_array2[:,2]) 
α₁ = median(chain_array2[:,3]) 
tspan = (0.0,last(time_to_fit2)+20)
p = (β₃, α₆, α₁)
probplot2 = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth2 = solve(probplot2,alg,saveat=0.1,callback=floor_event)

β₃ = median(chain_array3[:,1]) 
α₆ = median(chain_array3[:,2]) 
α₁ = median(chain_array3[:,3]) 
tspan = (0.0,last(time_to_fit3)+20)
p = (β₃, α₆, α₁)
probplot3 = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth3 = solve(probplot3,alg,saveat=0.1,callback=floor_event)

β₃ = median(chain_array4[:,1]) 
α₆ = median(chain_array4[:,2]) 
α₁ = median(chain_array4[:,3]) 
tspan = (0.0,last(time_to_fit4)+20)
p = (β₃, α₆, α₁)
probplot4 = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth4 = solve(probplot4,alg,saveat=0.1,callback=floor_event)

β₃ = median(chain_array5[:,1]) 
α₆ = median(chain_array5[:,2]) 
α₁ = median(chain_array5[:,3])
tspan = (0.0,last(time_to_fit5)+20)
p = (β₃, α₆, α₁)
probplot5 = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth5 = solve(probplot5,alg,saveat=0.1,callback=floor_event)

β₃ = median(chain_array6[:,1]) 
α₆ = median(chain_array6[:,2]) 
α₁ = median(chain_array6[:,3]) 
tspan = (0.0,last(time_to_fit6)+20)
p = (β₃, α₆, α₁)
probplot6 = ODEProblem(ode_system,u0,tspan,p)
solplotsmooth6 = solve(probplot6,alg,saveat=0.1,callback=floor_event)

In [ ]:
time_to_fit_plot1 = append!([0.0],time_to_fit1) 
tumor_to_fit_plot1 = append!([0.0],tumor_to_fit1)

time_to_fit_plot2 = append!([0.0],time_to_fit2) 
tumor_to_fit_plot2 = append!([0.0],tumor_to_fit2) 

time_to_fit_plot3 = append!([0.0],time_to_fit3) 
tumor_to_fit_plot3 = append!([0.0],tumor_to_fit3) 

time_to_fit_plot4 = append!([0.0],time_to_fit4) 
tumor_to_fit_plot4 = append!([0.0],tumor_to_fit4) 

time_to_fit_plot5 = append!([0.0],time_to_fit5) 
tumor_to_fit_plot5 = append!([0.0],tumor_to_fit5) 

time_to_fit_plot6 = append!([0.0],time_to_fit6) 
tumor_to_fit_plot6 = append!([0.0],tumor_to_fit6)

In [ ]:
#plot fits
fig_layout = @layout [a b; c d; e f; g h; i j; k l]

plot()
for k in 1:100
    resol = solve(remake(prob1,p=chain_array1[rand(1:length(chain_array1[:,1])), 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    plot!(resol, alpha=0.1, color = "#BBBBBB", legend = false, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "")
end
a = plot!(solplotsmooth1, w=3, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (10^2,10^6), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

scatter()
traj1 = solve(remake(probplot1,p=chain_array1[1, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
array_of_traj1 = zeros(8000, length(traj1.t))
for k in 1:8000
    resol_tumor = solve(remake(probplot1,p=chain_array1[k, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
        for i = 1:convert(Int64, length(resol_tumor)) 
            array_of_traj1[k,i] = (resol_tumor[i][1]-u0[1])./u0[1]
        end
end
interval_low1 = zeros(length(traj1.t))
interval_high1 = zeros(length(traj1.t))
for k in 1:length(interval_low1)
    interval_low1[k] = quantile(array_of_traj1[:,k], 0.05)
    interval_high1[k] = quantile(array_of_traj1[:,k], 0.95)
end

predicted_data1 = ones(length(solplotsmooth1)) 
for i = 1:convert(Int64, length(solplotsmooth1))
    predicted_data1[i] = (solplotsmooth1[i][1]-u0[1])./u0[1]
end
plot!(solplotsmooth1.t, predicted_data1, ribbon = (abs.(interval_low1-predicted_data1), abs.(interval_high1-predicted_data1)),fillalpha= 0.1, linewidth = 5, xlims = (-5,last(time_to_fit_plot1)+5), size =(1600/2,600), font = "Ariel", color = logocolors.purple, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_to_fit_plot1, tumor_to_fit_plot1, xlims = (-5,last(time_to_fit_plot1)+5), size =(1600/2,600), font = "Ariel", color = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
b = scatter!(time_to_fit_plot1, tumor_to_fit_plot1, markersize = 10, xlims = (-5,last(time_to_fit_plot1)+5), size =(1600/2,600), font = "Ariel", markercolor = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

plot()
for k in 1:100
    resol = solve(remake(prob1,p=chain_array2[rand(1:length(chain_array2[:,1])), 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    plot!(resol, alpha=0.1, color = "#BBBBBB", legend = false, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "")
end
c = plot!(solplotsmooth2, w=3, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (10^2,10^6), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

scatter()
traj2 = solve(remake(probplot2,p=chain_array2[1, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
array_of_traj2 = zeros(8000, length(traj2.t))
for k in 1:8000
    resol_tumor = solve(remake(probplot2,p=chain_array2[k, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
        for i = 1:convert(Int64, length(resol_tumor)) 
            array_of_traj2[k,i] = (resol_tumor[i][1]-u0[1])./u0[1]
        end
end
interval_low2 = zeros(length(traj2.t))
interval_high2 = zeros(length(traj2.t))
for k in 1:length(interval_low2)
    interval_low2[k] = quantile(array_of_traj2[:,k], 0.05)
    interval_high2[k] = quantile(array_of_traj2[:,k], 0.95)
end

predicted_data2 = ones(length(solplotsmooth2)) 
for i = 1:convert(Int64, length(solplotsmooth2))
    predicted_data2[i] = (solplotsmooth2[i][1]-u0[1])./u0[1]
end
plot!(solplotsmooth2.t, predicted_data2, ribbon = (abs.(interval_low2-predicted_data2), abs.(interval_high2-predicted_data2)),fillalpha= 0.1, linewidth = 5, xlims = (-5,last(time_to_fit_plot2)+5), size =(1600/2,600), font = "Ariel", color = logocolors.purple, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_to_fit_plot2, tumor_to_fit_plot2, xlims = (-5,last(time_to_fit_plot2)+5), size =(1600/2,600), font = "Ariel", color = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
d = scatter!(time_to_fit_plot2, tumor_to_fit_plot2, markersize = 10, xlims = (-5,last(time_to_fit_plot2)+5), size =(1600/2,600), font = "Ariel", markercolor = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

plot()
for k in 1:100
    resol = solve(remake(prob1,p=chain_array3[rand(1:length(chain_array3[:,1])), 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    plot!(resol, alpha=0.1, color = "#BBBBBB", legend = false, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "")
end
e = plot!(solplotsmooth3, w=3, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (10^2,10^6), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

scatter()
traj3 = solve(remake(probplot3,p=chain_array3[1, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
array_of_traj3 = zeros(8000, length(traj3.t))
for k in 1:8000
    resol_tumor = solve(remake(probplot3,p=chain_array3[k, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
        for i = 1:convert(Int64, length(resol_tumor)) 
            array_of_traj3[k,i] = (resol_tumor[i][1]-u0[1])./u0[1]
        end
end
interval_low3 = zeros(length(traj3.t))
interval_high3 = zeros(length(traj3.t))
for k in 1:length(interval_low3)
    interval_low3[k] = quantile(array_of_traj3[:,k], 0.05)
    interval_high3[k] = quantile(array_of_traj3[:,k], 0.95)
end

predicted_data3 = ones(length(solplotsmooth3)) 
for i = 1:convert(Int64, length(solplotsmooth3))
    predicted_data3[i] = (solplotsmooth3[i][1]-u0[1])./u0[1]
end
plot!(solplotsmooth3.t, predicted_data3, ribbon = (abs.(interval_low3-predicted_data3), abs.(interval_high3-predicted_data3)),fillalpha= 0.1, linewidth = 5, xlims = (-5,last(time_to_fit_plot3)+5), size =(1600/2,600), font = "Ariel", color = logocolors.purple, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_to_fit_plot3, tumor_to_fit_plot3, xlims = (-5,last(time_to_fit_plot3)+5), size =(1600/2,600), font = "Ariel", color = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
f = scatter!(time_to_fit_plot3, tumor_to_fit_plot3, markersize = 10, xlims = (-5,last(time_to_fit_plot3)+5), size =(1600/2,600), font = "Ariel", markercolor = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

plot()
for k in 1:100
    resol = solve(remake(prob1,p=chain_array4[rand(1:length(chain_array4[:,1])), 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    plot!(resol, alpha=0.1, color = "#BBBBBB", legend = false, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "")
end
g = plot!(solplotsmooth4, w=3, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (10^2,10^6), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

scatter()
traj4 = solve(remake(probplot4,p=chain_array4[1, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
array_of_traj4 = zeros(8000, length(traj4.t))
for k in 1:8000
    resol_tumor = solve(remake(probplot4,p=chain_array4[k, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
        for i = 1:convert(Int64, length(resol_tumor)) 
            array_of_traj4[k,i] = (resol_tumor[i][1]-u0[1])./u0[1]
        end
end
interval_low4 = zeros(length(traj4.t))
interval_high4 = zeros(length(traj4.t))
for k in 1:length(interval_low4)
    interval_low4[k] = quantile(array_of_traj4[:,k], 0.05)
    interval_high4[k] = quantile(array_of_traj4[:,k], 0.95)
end

predicted_data4 = ones(length(solplotsmooth4)) 
for i = 1:convert(Int64, length(solplotsmooth4))
    predicted_data4[i] = (solplotsmooth4[i][1]-u0[1])./u0[1]
end
plot!(solplotsmooth4.t, predicted_data4, ribbon = (abs.(interval_low4-predicted_data4), abs.(interval_high4-predicted_data4)),fillalpha= 0.1, linewidth = 5, xlims = (-5,last(time_to_fit_plot4)+5), size =(1600/2,600), font = "Ariel", color = logocolors.purple, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_to_fit_plot4, tumor_to_fit_plot4, xlims = (-5,last(time_to_fit_plot4)+5), size =(1600/2,600), font = "Ariel", color = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
h = scatter!(time_to_fit_plot4, tumor_to_fit_plot4, markersize = 10, xlims = (-5,last(time_to_fit_plot4)+5), size =(1600/2,600), font = "Ariel", markercolor = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

plot()
for k in 1:100
    resol = solve(remake(prob1,p=chain_array5[rand(1:length(chain_array5[:,1])), 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    plot!(resol, alpha=0.1, color = "#BBBBBB", legend = false, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "")
end
iplot = plot!(solplotsmooth5, w=3, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (10^2,10^6), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

scatter()
traj5 = solve(remake(probplot5,p=chain_array5[1, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
array_of_traj5 = zeros(8000, length(traj5.t))
for k in 1:8000
    resol_tumor = solve(remake(probplot5,p=chain_array5[k, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
        for i = 1:convert(Int64, length(resol_tumor)) 
            array_of_traj5[k,i] = (resol_tumor[i][1]-u0[1])./u0[1]
        end
end
interval_low5 = zeros(length(traj5.t))
interval_high5 = zeros(length(traj5.t))
for k in 1:length(interval_low5)
    interval_low5[k] = quantile(array_of_traj5[:,k], 0.05)
    interval_high5[k] = quantile(array_of_traj5[:,k], 0.95)
end

predicted_data5 = ones(length(solplotsmooth5)) 
for i = 1:convert(Int64, length(solplotsmooth5))
    predicted_data5[i] = (solplotsmooth5[i][1]-u0[1])./u0[1]
end
plot!(solplotsmooth5.t, predicted_data5, ribbon = (abs.(interval_low5-predicted_data5), abs.(interval_high5-predicted_data5)),fillalpha= 0.1, linewidth = 5, xlims = (-5,last(time_to_fit_plot5)+5), size =(1600/2,600), font = "Ariel", color = logocolors.purple, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_to_fit_plot5, tumor_to_fit_plot5, xlims = (-5,last(time_to_fit_plot5)+5), size =(1600/2,600), font = "Ariel", color = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
j = scatter!(time_to_fit_plot5, tumor_to_fit_plot5, markersize = 10, xlims = (-5,last(time_to_fit_plot5)+5), size =(1600/2,600), font = "Ariel", markercolor = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)

plot()
for k in 1:100
    resol = solve(remake(prob1,p=chain_array6[rand(1:length(chain_array6[:,1])), 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u))
    plot!(resol, alpha=0.1, color = "#BBBBBB", legend = false, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (0.01,10^7.5), xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "")
end
kplot = plot!(solplotsmooth6, w=3, yaxis=:log,size =(1600/2,600), font = "Ariel", ylims = (10^2,10^6), color = [logocolors.red "goldenrod2" logocolors.green logocolors.blue], alpha = [1 1 0.3 0.3], legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Number of cells",yguidefontsize=25,title = "",titlefontsize=25)

scatter()
traj6 = solve(remake(probplot6,p=chain_array6[1, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
array_of_traj6 = zeros(8000, length(traj6.t))
for k in 1:8000
    resol_tumor = solve(remake(probplot6,p=chain_array6[k, 1:4]),Tsit5(),saveat=0.1,callback=floor_event,reltol=1e-6,isoutofdomain=(u,p,t)->any(x->x<0,u)) 
        for i = 1:convert(Int64, length(resol_tumor)) 
            array_of_traj6[k,i] = (resol_tumor[i][1]-u0[1])./u0[1]
        end
end
interval_low6 = zeros(length(traj6.t))
interval_high6 = zeros(length(traj6.t))
for k in 1:length(interval_low6)
    interval_low6[k] = quantile(array_of_traj6[:,k], 0.05)
    interval_high6[k] = quantile(array_of_traj6[:,k], 0.95)
end

predicted_data6 = ones(length(solplotsmooth6)) 
for i = 1:convert(Int64, length(solplotsmooth6))
    predicted_data6[i] = (solplotsmooth6[i][1]-u0[1])./u0[1]
end
plot!(solplotsmooth6.t, predicted_data6, ribbon = (abs.(interval_low6-predicted_data6), abs.(interval_high6-predicted_data6)),fillalpha= 0.1, linewidth = 5, xlims = (-5,last(time_to_fit_plot6)+5), size =(1600/2,600), font = "Ariel", color = logocolors.purple, legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
plot!(time_to_fit_plot6, tumor_to_fit_plot6, xlims = (-5,last(time_to_fit_plot6)+5), size =(1600/2,600), font = "Ariel", color = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)
l = scatter!(time_to_fit_plot6, tumor_to_fit_plot6, markersize = 10, xlims = (-5,last(time_to_fit_plot6)+5), size =(1600/2,600), font = "Ariel", markercolor = "Black", legend = false, xtickfontsize=25,ytickfontsize=25,xlabel="Time (days)",xguidefontsize=25,ylabel="Relative change",yguidefontsize=25,title = "",titlefontsize=25)


In [ ]:
plot(a, b, c, d, e, f, g, h, iplot, j, kplot, l, layout = fig_layout, size =(1600,1200*2), plot_title ="", titlefontsize=25)

# Decision trees to classify tumor response (increasing or decreasing)

In [ ]:
using DecisionTree
using JLD

In [ ]:
chain_data

In [ ]:
only_two = [chain_data[:,1] chain_data[:,3]]

In [ ]:
chain_classification = Vector{String}(undef, length(chain_data[:,1]))
for i = 1:16000
    chain_classification[i] = "decreasing"
end
for i = 16001:32000
    chain_classification[i] = "increasing"
end

In [ ]:
#run decision tree on parameters from MCMC posteriors
features = float.(only_two)
labels   = string.(chain_classification)

In [ ]:
# train depth-truncated classifier
model = DecisionTreeClassifier(max_depth=3)

In [ ]:
#fit model
fit!(model, features, labels)

In [ ]:
#print tree
print_tree(model, 3)

In [ ]:
#apply model
DecisionTree.predict(model, [2.30843e-5, 0.74183])

In [ ]:
predict_proba(model, [2.30843e-5, 0.74183])

In [ ]:
println(get_classes(model))